In [2]:
import pandas as pd
import numpy as np

In [3]:
raw_df = pd.read_excel('EGAD PFAS Statewide File November 2023 V1-1f.xlsx', sheet_name='PFAS Sample Data', skiprows=1)

In [12]:
#rename columns
df = raw_df.copy()
df.columns = df.columns.str.replace(" ", "_").str.lower()
#map the acid versions "_A" back to original parameter
df["parameter"] = df["parameter"].str.replace("_A$", "", regex=True)
#convert town and site names to have title capitalization
df["town"] = df["town"].str.title()
df["current_site_name"] = df["current_site_name"].str.title()

#only keep rows with readings
df = df.dropna(subset=["concentration"])
#get rid of misc columns
df = df.drop(['unnamed:_0', 'lab_qualifier', 'rl', 'sample_point_seq'], axis=1)

#convert units
df.loc[(df["sample_type"] == "MLK") & (df["units"] == "ng/g"), "concentration"] *= 1000
df.loc[df["sample_type"] == "MLK", "units"] = "ng/L"
df.loc[(df["sample_type"] == "SL") & (df["units"] == "ng/L"), "concentration"] /= 1000
df.loc[df["sample_type"] == "SL", "units"] = "ng/g"

#combine SF and SOF
df.loc[df["sample_type"] == "SOF", "sample_type"] = "SF" 

In [13]:
# convert from sample type code to description
sample_key = pd.read_csv('sample_key.csv').set_index('code')
sample_key['description'] = sample_key['description'].str.title()
sample_lookup = sample_key['description'].to_dict()

# df['sample_type'] = df['sample_type'].apply(sample_lookup.get)

df = df.replace(to_replace={
                'ts': {'T': 'Treated', 'N': 'Not treated', 'NA': 'Not appicable', np.NAN: 'Unknown', 'U': 'Unknown'},
                'sample_type': sample_lookup
            }
)
#rename skinless filet to just filet (since we combined skinless and with skin)
df.loc[df["sample_type"] == "Skinless Filet", "sample_type"] = "Filet" 


In [14]:
# Define thresholds
thresholds = {
    "Drinking Water": {
        "PFOA": 4,
        "PFOS": 4,
        "PFHXS": 10,
        "HFPO-DA": 10,
        "PFNA": 10,
        "SUM OF 6 PFAS": 20,
    },
    "Soil": {
        "PFOA": 740,
        "PFOS": 490,
        "PFHXS": 4900,
        "PFNA": 740,
        "PFBS": 74000,
        "PFBA": 300000,
        "PFHXA": 120000,
    },
    "Milk": {"PFOS": 210},
    "Meat": {"PFOS": 3.4},
    "Filet": {"PFOS": 3.5},
}

# units = {"DW": "ng/L", "SL": "ng/g", "MLK": "ng/L", "MEA": "ng/g", "SF": "ng/g"}

In [15]:
def get_threshold_pct(row):
    sample_thresholds = thresholds.get(row['sample_type'])
    if sample_thresholds is None:
        return pd.NA
    
    param_threshold = sample_thresholds.get(row['parameter'])
    if param_threshold is None:
        return pd.NA
    
    return row['concentration'] / param_threshold * 100

In [16]:
df['threshold_pct'] = df.apply(get_threshold_pct, axis=1)

In [17]:
df.to_csv('cleaned_data.csv', index=False)

In [18]:
pd.read_csv('cleaned_data.csv')

,current_site_name,town,sample_date,sample_type,parameter,concentration,units,ts,threshold_pct
0,Unity Landfill,Unity,2022-11-03 13:13:00,Groundwater,N-EtFOSAA,1.400,ng/L,Unknown,NaN
1,Unity Landfill,Unity,2022-11-03 13:13:00,Groundwater,PFBA,1.560,ng/L,Unknown,NaN
2,Unity Landfill,Unity,2022-11-03 13:13:00,Groundwater,PFBS,0.364,ng/L,Unknown,NaN
3,Unity Landfill,Unity,2022-11-03 13:13:00,Groundwater,PFDA,0.455,ng/L,Unknown,NaN
4,Unity Landfill,Unity,2022-11-03 13:13:00,Groundwater,PFDOA,0.553,ng/L,Unknown,NaN
...,...,...,...,...,...,...,...,...,...
123399,Dexter Landfill,Dexter,2018-08-16 12:45:00,Pore Water,PFNA,5.020,ng/L,Unknown,NaN
123400,Dexter Landfill,Dexter,2018-08-16 12:45:00,Pore Water,PFOA + PFOS,501.000,ng/L,Unknown,NaN
123401,Dexter Landfill,Dexter,2018-08-16 12:45:00,Pore Water,PFOA,120.000,ng/L,Unknown,NaN
123402,Dexter Landfill,Dexter,2018-08-16 12:45:00,Pore Water,PFOS,381.000,ng/L,Unknown,NaN


In [19]:
max(pd.to_datetime(df['sample_date']))

Timestamp('2023-10-24 13:30:00')